In [1]:
import numpy as np
import cv2
import random

高斯噪声

In [3]:
# def GaussianNoise(img,means,sigma,percetage):
#     H,W=img.shape
#     NoiseNum = int(percetage*H*W)

    
#     for _ in range(NoiseNum):
#         randX=random.randint(0,H-1)
#         randY=random.randint(0,W-1)
#         img[randX,randY]+=random.gauss(means,sigma)
#         if img[randX,randY]<0:
#             img[randX,randY]=0
#         elif img[randX,randY]>255:
#             img[randX,randY]=255
#     return img

def GaussianNoise(src, means, sigma, percentage):
    NoiseImg = src.copy()
    NoiseNum = int(percentage * src.shape[0] * src.shape[1])
    
    # 生成所有可能的坐标
    coordinates = [(x, y) for x in range(src.shape[0]) for y in range(src.shape[1])]
    
    # 随机选择不重复的坐标
    selected_coordinates = random.sample(coordinates, NoiseNum)
    
    for (randX, randY) in selected_coordinates:
        # 在原有像素灰度值上加上随机数
        NoiseImg[randX, randY] = NoiseImg[randX, randY] + random.gauss(means, sigma)
        # 限制灰度值在0-255范围内
        NoiseImg[randX, randY] = np.clip(NoiseImg[randX, randY], 0, 255)
    
    return NoiseImg
        
    
img = cv2.imread('lenna.png',0)
cv2.imshow("lenna",img)

img_GaussianNoise = GaussianNoise(img,2,4,0.8)
cv2.imshow("img_GaussianNoise",img_GaussianNoise)

cv2.waitKey(0)

-1

椒盐噪声

In [4]:
def Salt_Pepper_Noise(img,percetage):
    H,W=img.shape
    NoiseNum = int(percetage*H*W)

    for _ in range(NoiseNum):
        randX=random.randint(0,H-1)
        randY=random.randint(0,W-1)
        if random.random()<=0.5:
            img[randX,randY]=0
        else:
            img[randX,randY]=255
    return img
        
    
img = cv2.imread('lenna.png',0)
cv2.imshow("lean",img)

img_Salt_Pepper_Noise = Salt_Pepper_Noise(img,0.8)
cv2.imshow("img_Salt_Pepper_Noise",img_Salt_Pepper_Noise)

cv2.waitKey(0)

-1

PCA

In [13]:
import numpy as np

class PCA(object):
    def __init__(self,X,k) -> None:
        self.X=X
        self.k=k
        
        '''矩阵X的中心化'''
        mean = np.array([np.mean(attr) for attr in self.X.T])
        print('样本集的特征均值:\n',mean)
        centrX = self.X - mean
        print('样本矩阵X的中心化centrX:\n', centrX)
        
        '''求样本矩阵X的协方差矩阵C'''
        ns = np.shape(centrX)[0]
        C = np.dot(centrX.T, centrX)/(ns - 1) # 无偏估计
        print('样本矩阵X的协方差矩阵C:\n', C)
        
        '''求X的降维转换矩阵U, shape=(n,k), n是X的特征维度总数，k是降维矩阵的特征维度'''
        a,b = np.linalg.eig(C)
        print('样本集的协方差矩阵C的特征值:\n', a)
        print('样本集的协方差矩阵C的特征向量:\n', b)
        ind = np.argsort(-1*a)
        UT = [b[:,ind[i]] for i in range(self.k)]
        U = np.transpose(UT)
        print('%d阶降维转换矩阵U:\n'%self.k, U)
        
        '''按照Z=XU求降维矩阵Z, shape=(m,k), n是样本总数，k是降维矩阵中特征维度总数'''
        Z = np.dot(self.X, U)
        print('X shape:', np.shape(self.X))
        print('U shape:', np.shape(U))
        print('Z shape:', np.shape(Z))
        print('样本矩阵X的降维矩阵Z:\n', Z)
        # return
        
        
        
        
X = np.array([[10, 15, 29],
            [15, 46, 13],
            [23, 21, 30],
            [11, 9,  35],
            [42, 45, 11],
            [9,  48, 5],
            [11, 21, 14],
            [8,  5,  15],
            [11, 12, 21],
            [21, 20, 25]])
k = np.shape(X)[1] - 1
print('样本集(10行3列，10个样例，每个样例3个特征):\n', X)
pca = PCA(X,k)

样本集(10行3列，10个样例，每个样例3个特征):
 [[10 15 29]
 [15 46 13]
 [23 21 30]
 [11  9 35]
 [42 45 11]
 [ 9 48  5]
 [11 21 14]
 [ 8  5 15]
 [11 12 21]
 [21 20 25]]
样本集的特征均值:
 [16.1 24.2 19.8]
样本矩阵X的中心化centrX:
 [[ -6.1  -9.2   9.2]
 [ -1.1  21.8  -6.8]
 [  6.9  -3.2  10.2]
 [ -5.1 -15.2  15.2]
 [ 25.9  20.8  -8.8]
 [ -7.1  23.8 -14.8]
 [ -5.1  -3.2  -5.8]
 [ -8.1 -19.2  -4.8]
 [ -5.1 -12.2   1.2]
 [  4.9  -4.2   5.2]]
样本矩阵X的协方差矩阵C:
 [[ 108.32222222   74.53333333  -10.08888889]
 [  74.53333333  260.62222222 -106.4       ]
 [ -10.08888889 -106.4          94.17777778]]
样本集的协方差矩阵C的特征值:
 [335.15738485  95.32771231  32.63712506]
样本集的协方差矩阵C的特征向量:
 [[-0.30253213 -0.87499307 -0.37797014]
 [-0.86718533  0.08811216  0.49012839]
 [ 0.39555518 -0.47604975  0.78543792]]
2阶降维转换矩阵U:
 [[-0.30253213 -0.87499307]
 [-0.86718533  0.08811216]
 [ 0.39555518 -0.47604975]]
X shape: (10, 3)
U shape: (3, 2)
Z shape: (10, 2)
样本矩阵X的降维矩阵Z:
 [[ -4.56200104 -21.2336912 ]
 [-39.28629002 -15.26038349]
 [-13.30247561 -32.55597794]
 [  